In [1]:
import sys
sys.path.append('../..') # just in case gp_grief isn't in the path
import numpy as np
from numpy.testing import assert_array_almost_equal, assert_array_equal
from time import time
import gp_grief.tensors
from gp_grief.tensors import *
import pdb

# `KronMatrix` Testing

test some basic kronecker product operations

In [2]:
for sym in [True, False]:
    print '\n' + '*' * 80
    print 'sym=%s\n' % (sym)
    np.random.seed(0)
    d = 3
    n = 5
    N = n**d
    sym = True
    A = [np.array(np.random.rand(n,n),order='F') for i in range(d)]
    if sym:
        A = [np.array(Ai.dot(Ai.T) + 1e-6*np.identity(n),order='F') for Ai in A]
    Ab = 1
    for i in range(d):
        Ab = np.kron(Ab,A[i])
    K = gp_grief.tensors.KronMatrix(A, sym=sym)
    x = np.matrix(np.random.rand(n**d,1))

    # test the expansion
    print "expansion"
    error = np.linalg.norm(K.expand()-Ab)/np.linalg.norm(Ab)
    print '\tRelative error is %g' % error
    if error > 1e-10:
        raise RuntimeError('error too large.')

    # test the transpose
    print "\ntranspose"
    error =  np.linalg.norm(K.T.expand()-Ab.T)/np.linalg.norm(Ab)
    print '\tRelative error is %g' % error
    if error > 1e-10:
        raise RuntimeError('error too large.')

    # test a matrix vector product
    print "\nvector product"
    error = np.linalg.norm(K*x-Ab.dot(x))/np.linalg.norm(Ab.dot(x))
    print '\tRelative error is %g' % error
    if error > 1e-10:
        raise RuntimeError('error too large.')
    t = time()
    K.kronvec_prod(x);
    print "\ttime for mvprod %.3g seconds" % (time()-t)

    # test solving a linear system
    print "\nlinear system solve"
    error = np.linalg.norm(Ab.dot(K.kronvec_div(x))-x)/np.linalg.norm(x)
    error = max(error, np.linalg.norm(K*(K.kronvec_div(x))-x)/np.linalg.norm(x)) # verify consistency
    print '\tRelative error is %g' % error
    if error > 1e-10:
        raise RuntimeError('error too large.')

    # test chol
    print "\ncholesky decomposition"
    C = K.chol()
    error =  np.linalg.norm((C.T).kronkron_prod(C).expand() - Ab)/np.linalg.norm(Ab) # try and reconstruct K
    error = max(error, np.linalg.norm(K*(C.solve_chol(x))-x)/np.linalg.norm(x)) # solve linear system
    print '\tRelative error is %g' % error
    if error > 1e-10:
        raise RuntimeError('error too large.')

    # test schur
    print "\nschur decomposition"
    Q,T = K.schur()
    error = np.linalg.norm(Q.kronkron_prod(T).kronkron_prod(Q.T).expand() - Ab)/np.linalg.norm(Ab) # try and reconstruct K
    error = max(error, np.linalg.norm(K*(Q.solve_schur(T,x))-x)/np.linalg.norm(x)) # solve linear system
    lam = 1e-3
    y = Q.solve_schur(T,x,shift=lam)
    error = max(error, np.linalg.norm(K*y+lam*y-x)/np.linalg.norm(x)) # solve a shifted linear system
    print '\tRelative error is %g' % error
    if error > 1e-10:
        raise RuntimeError('error too large.')

    # test svd
    print "\nsvd"
    Q,eig_vals = K.svd()
    # reconstruct K
    assert_array_almost_equal(Q.expand().dot(np.diag(eig_vals.expand()).dot(Q.T.expand())), Ab)
    # solve shifted linear system
    y = Q.solve_schur(eig_vals.expand(),x,shift=lam)
    assert_array_almost_equal(K*y+lam*y, x)
print "*"*80
print "tests passed!"


********************************************************************************
sym=True

expansion
	Relative error is 0

transpose
	Relative error is 0

vector product
	Relative error is 1.45337e-16
	time for mvprod 0.000125 seconds

linear system solve
	Relative error is 3.3406e-11

cholesky decomposition
	Relative error is 4.36564e-12

schur decomposition
	Relative error is 1.23528e-11

svd

********************************************************************************
sym=False

expansion
	Relative error is 0

transpose
	Relative error is 0

vector product
	Relative error is 1.45337e-16
	time for mvprod 4.51e-05 seconds

linear system solve
	Relative error is 3.3406e-11

cholesky decomposition
	Relative error is 4.36564e-12

schur decomposition
	Relative error is 1.23528e-11

svd
********************************************************************************
tests passed!


Test the eigenvalue/vector sorting to get the largest and smallest

In [3]:
np.random.seed(1)
d = 10
n = 3
eigs = gp_grief.tensors.KronMatrix([np.random.rand(n) for i in range(d)])
all_eigs = eigs.expand() # compute all the eigenvalues for comparison
n_eigs = 5 # this is the number of largest/smallest that I want to find
for log_expand in [False,True]:
    for mode in ['largest', 'smallest']:
        # get the n_eigs largest/smallest
        eig_order, extreme_eigs, global_loc = eigs.find_extremum_eigs(n_eigs,mode=mode,log_expand=log_expand,
                                                                      sort=True, compute_global_loc=True)
        if log_expand: # transform back from log space
            extreme_eigs = np.exp(extreme_eigs)

        # check if extreme_eigs is being computed correctly
        assert_array_almost_equal(extreme_eigs, 
            [np.prod([eigs.K[i][eig_order[j,i]] for i in range(d)]) for j in range(n_eigs)])

        # ensure global_loc was computed correctly
        assert_array_almost_equal(extreme_eigs, all_eigs[global_loc], decimal=15)

        # then compare with the brute force expansion to ensure the correct values were selected
        if mode == 'largest':
            extreme_eigs_exact = np.sort(all_eigs)[::-1][:n_eigs]
        elif mode == 'smallest':
            extreme_eigs_exact = np.sort(all_eigs)[:n_eigs]
        assert_array_almost_equal(extreme_eigs[::-1], np.sort(extreme_eigs_exact),decimal=15) 
print 'done tests.'

done tests.


test the log determinant

In [4]:
for sym in [True,False]:
    np.random.seed(0)
    A = [np.random.rand(5,5)+np.eye(5) for i in range(2)]
    A = [Ai.dot(Ai.T)+1e-6*np.eye(5) for Ai in A] # make it SPD
    A = gp_grief.tensors.KronMatrix(A,sym=sym)
    eig_vals = A.eig_vals()
    assert_array_almost_equal(eig_vals.log_det(), np.linalg.slogdet(A.expand())[1])
print 'done tests.'

done tests.


test flipping/shuffling the matrix-multiplication order.

*Note that this isn't so much a test of the `tensors` library as it is for future reference to recall how to do this.*

In [5]:
np.random.seed(0)
shapes = [(2,3), (2,2), (5,2)] # sizes of submatricies
d = len(shapes)

# first do the exact computation
K = gp_grief.tensors.KronMatrix([np.random.rand(*shape) for shape in shapes])
x = np.random.rand(K.shape[1], 1)
y = K*x

# now shuffle K and the vector x and try to recover y
for i in range(1,d): # i is the index which should go first
    # do the forward shuffle
    shuffle = np.concatenate(([i,], np.delete(np.arange(d), i)))
    print "Testing shuffle order %s" % shuffle
    K_s = gp_grief.tensors.KronMatrix([K.K[axis] for axis in shuffle]) # shuffled kronecker product
    X = x.reshape(zip(*shapes)[1]) # reshape x to the grid shape
    x_s = np.transpose(X, shuffle).reshape((-1,1)) # shuffle and turn back to vector
    y_s = K_s * x_s
    
    # now reverse the shuffle in y
    new_shapes = [shapes[j] for j in shuffle] # shuffled shape of grid
    reverse = np.squeeze([np.where(shuffle==j)[0] for j in range(d)]) # order of the reverse shuffle
    Y_s = y_s.reshape(zip(*new_shapes)[0]) # reshape y_s to the new (shuffled) grid shape
    yy = np.transpose(Y_s, reverse).reshape((-1,1)) # reverse shuffle and turn back to vector
    assert_array_almost_equal(yy,y)
print 'done tests.'

Testing shuffle order [1 0 2]
Testing shuffle order [2 0 1]
done tests.


# `SelectionMatrix` Testing

In [6]:
np.random.seed(0)
A = np.random.rand(20,20)
sel = np.random.choice(A.shape[0], size=30)

# check SelectionMatrix
S = gp_grief.tensors.SelectionMatrix((sel, A.shape[0]))
assert_array_equal(A[sel], S.mul(A)) # check if able to perform simple subset

# check SelectionMatrixSparse
S = gp_grief.tensors.SelectionMatrixSparse((sel, A.shape[0]))
assert_array_equal(A[sel], S.mul(A)) # check if able to perform simple subset

# check if able to perform unique subset then expand
assert_array_equal(A[sel], S.mul_unique(A)[S.unique_inverse])
print "done tests."

done tests.


# `BlockMatrix` Testing

In [7]:
np.random.seed(0)

# initialize random matricies
a = np.random.rand(2,3); b = np.random.rand(2,2);
c = np.random.rand(3,3); d = np.random.rand(3,2);
A = np.vstack((np.hstack((a,b)), np.hstack((c,d)))); 
Ablock = gp_grief.tensors.BlockMatrix(A=np.array([[Array(a),Array(b)],[Array(c),Array(d)]])); 

# test the transpose and expansion operations
assert_array_almost_equal(A,  Ablock.expand()  , decimal=8)
assert_array_almost_equal(A.T,Ablock.T.expand(), decimal=8)

# initialize random vectors
x = np.random.rand(A.shape[1],1); 
z = np.random.rand(A.shape[0],1);

# test matrix vector products
assert_array_almost_equal(A.dot(x),   Ablock  *x, decimal=8)
assert_array_almost_equal(A.T.dot(z), Ablock.T*z, decimal=8)

print 'done tests.'

done tests.


# `KhatriRaoMatrix` Testing
We will test here matrix-vector products with a row partitioned Khatri-Rao product matrix and its transpose

In [8]:
np.random.seed(0)
n_rows = 5
n_cols = (2,3,5)
partition = 0 # row partitioned

# generate random matricies and initialize Khatri-Rao Matrix
Araw = np.empty(len(n_cols),dtype=object)
Araw[:] = [np.random.rand(n_rows,nc) for nc in n_cols]
Akr = gp_grief.tensors.KhatriRaoMatrix(A=Araw, partition=partition)

# expand the Khatri-Rao matrix to use for testing
Abig = Akr.expand()

# initialize randome vectors for matrix vector products
x = np.random.rand(Abig.shape[1],1); 
z = np.random.rand(Abig.shape[0],1);

# test matrix vector products
assert_array_almost_equal(Abig.dot(x),   Akr  *x, decimal=8)
assert_array_almost_equal(Abig.T.dot(z), Akr.T*z, decimal=8)

print 'done tests.'

done tests.


# `RowColKhatriRaoMatrix` Testing
use `KhatriRaoMatrix` and `KronMatrix` to test this

In [9]:
# get random matricies: the overall matrix will have shape (p, N)
np.random.seed(0)
N = 5
p = 6 
d = 3
grid_shape = np.random.randint(low=2,high=15,size=d)
R = np.empty(d,dtype=object)
K = np.empty(d,dtype=object)
C = np.empty(d,dtype=object)
R[:] = [np.random.rand(p,m)-0.5 for m in grid_shape]
K[:] = [np.random.rand(m,m)-0.5 for m in grid_shape]
C[:] = [np.random.rand(m,N)-0.5 for m in grid_shape]
for i in range(d):
    R[i][0,:] = 0. # set this to zero so there's a zero in the final matrix
vec = np.random.rand(N,1)-0.5
vecT = np.random.rand(p,1)-0.5

# initialize RowKronColKhatriRaoMatrix
A = RowColKhatriRaoMatrix(R=R,K=K,C=C)

# initialize RowColKhatriRaoMatrixTransposed
AT = RowColKhatriRaoMatrixTransposed(R=R,K=K,C=C)

# initialize KhatriRaoMatrix's and KronMatrix to test
R = KhatriRaoMatrix(R,partition=0)
C = KhatriRaoMatrix(C,partition=1)
K = KronMatrix(K)

# test matvec
assert_array_almost_equal(A*vec, R*(K*(C*vec)))

# test matvec with transpose
assert_array_almost_equal(A.T*vecT, C.T*(K.T*(R.T*vecT)))

# now try with RowColKhatriRaoMatrixTransposed
assert_array_almost_equal(AT*vecT, C.T*(K.T*(R.T*vecT)))

# test the expand method to compute the whole matrix
RKC = R.expand().dot(K.expand().dot(C.expand()))
assert_array_almost_equal(A.expand(), RKC)

# test the log expand
log_A, sign = A.expand(logged=True)
assert_array_almost_equal(sign*np.exp(log_A), RKC)

print 'done tests.'

done tests.


# `TensorProduct` Testing
Test matrix-vector product
$$ABCDv$$

In [10]:
np.random.seed(0)
A =Array(np.random.rand(5,3))
B =Array(np.random.rand(3,8))
C =Array(np.random.rand(8,16))
D =Array(np.random.rand(16,12))
vec = np.random.rand(12,1)
exact_arr = A*(B*(C*D.A))
arr = gp_grief.tensors.TensorProduct([A,B,C,D])
assert_array_almost_equal(exact_arr.dot(vec), arr*vec, decimal=8)

print 'done tests.'

done tests.
